In [1]:
import requests
from datetime import date
import ryan
import fire

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [2]:
# Use a service account.
cred = credentials.Certificate('flightsAccount.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
#matching destinations
def matchDestinations(Origin, Arrival):
    matchingDestinations = []
    for destination in Origin:
        if destination in Arrival:
            matchingDestinations.append(d)
    return matchingDestinations

In [4]:
#imput:
trip = 'VNO-PSR'
Outbound = '2023-09-09'
Return = '2023-09-30'

In [5]:
#checking for destinations and airports
checkDate = int(str(date.today()).replace('-',''))
for origin in trip.split('-'):
    fireBatch = db.batch()
    #API call for destinations
    destinations = requests.get(ryan.getDestinations(origin,2)).json()
    #read response, set batch for Firestore and create array of destinations 
    rDestinations = ryan.readDestinations(destinations, fireBatch, db)
    #commiting batch
    fireBatch.commit()

    #create or update destinations from airport
    fire.fireDestinations(db, origin, rDestinations)

    #get array of matching destinations
    if origin == trip[:3]:
        rOriginDestinations = rDestinations
    else:
        rArrivalDestinations = rDestinations


#array for matched destinations
rConnections = []
if trip[:3] in rArrivalDestinations:
    isDirect = True
else:
    isDirect = False
for d in rOriginDestinations:
    if d in rArrivalDestinations:
        rConnections.append(d)
    
fire.fireConnections(db, trip, rConnections, isDirect)
fire.fireTrip(db, trip, int(str(date.today()).replace('-','')), True, rConnections, isDirect, Outbound.replace('-',''), Return.replace('-',''))
# fire.fireTrip(db, trip, int(str(date.today()).replace('-','')), True, rConnections, isDirect, Outbound.replace('-',''))

In [6]:
for i in rConnections:
    url = ryan.getFlights(i, trip[:3], Outbound, Return)
    print(url)

https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=BGY&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2023-09-09&FlexDaysOut=6&DateIn=2023-09-30&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=HHN&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2023-09-09&FlexDaysOut=6&DateIn=2023-09-30&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=MLA&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2023-09-09&FlexDaysOut=6&DateIn=2023-09-30&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&Destination=STN&INF=0&Origin=VNO&TEEN=0&IncludeConnectingFlights=false&DateOut=2023-09-09&FlexDaysOut=6&DateIn=2023-09-30&FlexDaysIn=6&RoundTrip=true&ToUs=AGREED
https://www.ryanair.com/api/booking/v4/en-en/availability?ADT=1&CHD=0&De

In [7]:
headers = {'Cookie': 'rid=53f81685-dd59-43a6-b829-2ac46e936cfa; rid.sig=OCBtG/WF9HO7H4S44kdr0UpsAs/uDH/Z89AjODYV4ipIZy+AD71pUFlLzf5wzEzBcRyxAYtAFGAtx4qTR8QVkUn51bdpJ6adVE+jcjrjjI+gS4t1gxaDdvBZs4Or2Dqxo/HarouZ+NRwtOYrk1d1WKoAbvkPW4Tr/OxK37O7JzNEQfTj/NYjObt4Vn8oTIij4S7BJ6f24RHOKf3Y2y1Gix27jFbvUe5Mh+5Pg4el5c309TyCtbIvycl9N1tcPeYXv7dDnk4KuGcxzl4zoVtTRXMh8O4JEr7shHjgB995Vpd6nJEJpVdtgYqGP51NK7djMcZB+HdJvDmgVI8556bYQxZle072SBcjGJgHC4k7a2spEA0OeaH1gG1gOzpODL7pubJzExfWNrrV94hudE3SKOY4S3wFdyNs6GB8QHwq/okZ+Xf+OEDZyhNAWDTMyZuzxoNPDME98mfIBDTRCuHeX1/G5ufUCC1G+qf0CBFoNhu8hXYOsfs/Bdh3ZkX92zTS0HeerO8LXCg4C9hpgq+g16tRewd1IdaOXRHmtVdMizw=; _ga=GA1.2.185346392.1675256153; STORAGE_PREFERENCES={"STRICTLY_NECESSARY":true,"PERFORMANCE":true,"FUNCTIONAL":true,"TARGETING":true,"SOCIAL_MEDIA":true,"PIXEL":true,"GANALYTICS":true,"__VERSION":2}; RY_COOKIE_CONSENT=true; bid_FRwdAp7a9G2cnLnTsgyBNeduseKcPcRy=8d7200bc-fe65-4a35-828c-05c5db690b69; _hjSessionUser_135144=eyJpZCI6ImYwNTcyYWYyLTVhMDYtNTg0My1hNzBkLWNjMTNjOTM5M2M4NiIsImNyZWF0ZWQiOjE2NzUyNTYxNTY1NzUsImV4aXN0aW5nIjp0cnVlfQ==; agsd=BXFqu0zsEJw9FQlrZw94l5ocpHvV_o6gxW4zfcOak1tJRf33; _cc=AcNukn4KxyCeXwwCm54JSS4K; fr-correlation-id=3c07d844-3f72-46a1-aaea-d81c822ba5e2; fr-correlation-id.sig=NNzxo5fz1Lzxa66hZof7b0c5szc; tj_seed=0025e97ccc74a598f297a6b2bdaf000000; __zlcmid=1F9liBTdl5vMDMU; _hjSessionUser_2481828=eyJpZCI6IjFhZjg1ODI5LTkwZjUtNTA3ZC1iNTVhLTg2YzJkNjkzNDViNiIsImNyZWF0ZWQiOjE2ODAyNTcxNzY5NjEsImV4aXN0aW5nIjpmYWxzZX0=; mkt=/gb/en/; _gid=GA1.2.161670609.1685688242; myRyanairID=; .AspNetCore.Session=CfDJ8Cxd1z5BGy5NmH8fAYZlb1QxQ80qqUfxxfQ0Ce7BOk%2FNZ7BhISvihHFiWzKXZvIgJG%2FDmB1Jjuo8gU9so9BmOcma7t3zeVdF%2FISPfGtsYYO96Jq8umNDC9mKclqpp%2BPqQbnlCl2OvfIUtOl7XAyfhQOC93VWLZ1Mno9lZYV8s%2BLm',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

ryanFlight = requests.get(ryan.getFlights('PSR', 'BGY', Outbound, Return), headers=headers).json()

In [14]:
for f in ryanFlight['trips']:
    # print(f"{f['origin']}-{f['destination']}")
    for d in f['dates']:
        # print(f"{f['origin']}-{f['destination']}", d['dateOut'].split('T')[0].replace('-',''))
        if d['flights'] != []:
            for flight in d['flights']:

                # print(f"{f['origin']}-{f['destination']}", d['dateOut'].split('T')[0].replace('-',''))
                # print(d['dateOut'], ''.join(d['flights'][0]['flightKey'].split()[0].split('~')[0:2]), d['flights'][0]['operatedBy'], d['flights'])
                # print(d['dateOut'], d['flights'][0]['duration'])
                # print(d['dateOut'], d['flights'][0]['regularFare']['fares'])
                print(f"{f['origin']}-{f['destination']}", flight['time'][0])

BGY-PSR ['2023-09-09T08:20:00.000', '2023-09-09T09:30:00.000']
BGY-PSR ['2023-09-09T16:20:00.000', '2023-09-09T17:30:00.000']
BGY-PSR ['2023-09-10T06:40:00.000', '2023-09-10T07:50:00.000']
BGY-PSR ['2023-09-10T22:20:00.000', '2023-09-10T23:30:00.000']
BGY-PSR ['2023-09-11T06:45:00.000', '2023-09-11T07:55:00.000']
BGY-PSR ['2023-09-12T07:05:00.000', '2023-09-12T08:15:00.000']
BGY-PSR ['2023-09-12T22:05:00.000', '2023-09-12T23:15:00.000']
BGY-PSR ['2023-09-13T07:00:00.000', '2023-09-13T08:10:00.000']
BGY-PSR ['2023-09-13T21:40:00.000', '2023-09-13T22:50:00.000']
BGY-PSR ['2023-09-14T05:45:00.000', '2023-09-14T06:55:00.000']
BGY-PSR ['2023-09-14T16:25:00.000', '2023-09-14T17:35:00.000']
BGY-PSR ['2023-09-15T18:05:00.000', '2023-09-15T19:15:00.000']
PSR-BGY ['2023-09-30T06:45:00.000', '2023-09-30T07:55:00.000']
PSR-BGY ['2023-09-30T17:55:00.000', '2023-09-30T19:05:00.000']
PSR-BGY ['2023-10-01T08:15:00.000', '2023-10-01T09:25:00.000']
PSR-BGY ['2023-10-01T23:55:00.000', '2023-10-02T01:05:0

In [20]:
fireBatch = db.batch()
for f in ryanFlight['trips']:
    for d in f['dates']:
        if d['flights'] != []:
            for flight in d['flights']:
                for time in flight['time']:
                    ref = db.collection('FlightsDev').document(f"{f['origin']}-{f['destination']}").collection('FlightDates').document(f"{f['origin']}-{f['destination']}_{d['dateOut'].split('T')[0].replace('-','')}")
                    fireBatch.set(ref, {
                        'OriginAirport': f['origin'],
                        'DestinationAirport': f['destination'],
                        'Date': d['dateOut'].split('T')[0].replace('-',''),
                        'FlightNumber': flight['flightNumber'].replace(' ',''),
                        'OperatedBy': flight['operatedBy'],
                        'LiftoffLocalTimeFull': flight['time'][0],
                        'LiftoffLocalTimeHour': flight['time'][0].split('T')[1].split(':')[0],
                        'LiftoffLocalTimeMinute': flight['time'][0].split('T')[1].split(':')[1],
                        'LandingLocalTimeFull': flight['time'][1],
                        'LandingLocalTimeHour': flight['time'][1].split('T')[1].split(':')[0],
                        'LandingLocalTimeMinute': flight['time'][1].split('T')[1].split(':')[1],
                        'LiftoffUTCTimeFull': flight['timeUTC'][0],
                        'LiftoffUTCTimeHour': flight['timeUTC'][0].split('T')[1].split(':')[0],
                        'LiftoffUTCTimeMinute': flight['timeUTC'][0].split('T')[1].split(':')[1],
                        'LandingUTCTimeFull': flight['timeUTC'][1],
                        'LandingUTCTimeHour': flight['timeUTC'][1].split('T')[1].split(':')[0],
                        'LandingUTCTimeMinute': flight['timeUTC'][1].split('T')[1].split(':')[1],
                        'Duration': flight['duration']
                    })
                    ref2 = db.collection('FlightsDev').document(f"{f['origin']}-{f['destination']}").collection('FlightDates').document(f"{f['origin']}-{f['destination']}_{d['dateOut'].split('T')[0].replace('-','')}").collection('FlightFares').document(str(date.today()).replace('-',''))
                    fireBatch.set(ref2, {
                        'CheckDate': int(str(date.today()).replace('-','')),
                        'FaresLeft': flight['faresLeft'],
                        'FlightKey': flight['flightKey'],
                        'FareClass': flight['regularFare']['fareClass'],
                        'FareType': flight['regularFare']['fares'][0]['type'],
                        'Fare': flight['regularFare']['fares'][0]['amount'],
                        'PublishedFare': flight['regularFare']['fares'][0]['publishedFare'],
                        'HasDiscount': flight['regularFare']['fares'][0]['hasDiscount'],
                        'HasPromoDiscount': flight['regularFare']['fares'][0]['hasPromoDiscount'],
                        'DiscountInPercent': flight['regularFare']['fares'][0]['discountInPercent'],
                        'DiscountAmount': flight['regularFare']['fares'][0]['discountAmount'],
                        'HasBogof': flight['regularFare']['fares'][0]['hasBogof'],
                        'FaresInfantsLeft': flight['infantsLeft']
                    })
    fireDoc = db.collection('FlightsDev').document(f"{f['origin']}-{f['destination']}")
    fireDoc.set({
        'LatestCheckDate': int(str(date.today()).replace('-',''))
    })
fireBatch.commit()

[update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_time {
   seconds: 1686556587
   nanos: 126330000
 },
 update_